In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *
from sympy import Matrix, symbols, solve_linear_system
from sympy.solvers.solveset import linsolve
import time
from joblib import Parallel, delayed
a,b,b1 = symbols('a b b_1')
x,y,z_1,z_2,z_3 = symbols('x y z_1 z_2 z_3')
from IPython.display import display, Latex

In [2]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals
from tensorly.decomposition import non_negative_parafac

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.0001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print('doing', k)
                # print(full)
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    return False


In [3]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 30
    a = np.random.randint(1, max, size=3) 
    b = np.random.randint(1, max, size=2)
    c = np.random.randint(1, max, size=2)
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

def low_tensor_spec():
    max = 300
    a = np.random.randint(1, max, size=2) * 0.1
    b = np.random.randint(1, max, size=3)* 0.1
    c = np.random.randint(1, max, size=3)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2 )) * 1.0
    return tens

def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t3 = tl.tensor([t[0], t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    a5 = det(Matrix(t3[0]))
    a6 = det(Matrix(t3[1]))

    b5 = det(Matrix(t3[:,0]))
    b6 = det(Matrix(t3[:,1]))

    c5 = det(Matrix(t3[:,:,0]))
    c6 = det(Matrix(t3[:,:,1]))
    return sgn([a1, a2,a3,a4,a5,a6]) or sgn([b1,b2,b3,b4,b5,b6]) or sgn([c1,c2,c3,c4,c5,c6])

def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)

def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod

def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

def check_r4(t):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return a1 or a2 or a3

def check_sub(t, i, j):
    temp = np.copy(t)
    temp[0][i][j] = 0
    temp[1][i][j] = 0
    temp[2][i][j] = 0
    t1 = tl.tensor([temp[0],temp[1]])
    t2 = tl.tensor([temp[0],temp[2]])
    t3 = tl.tensor([temp[1],temp[2]])
    return check_r2(t1) and check_r2(t2) and check_r2(t3)
    

In [10]:
def proc(i):
    tens = tl.tensor(np.random.randint(100, 2000000, size=(3,2,2))) * 0.01
    #tens = rank_tree()
    if True or check_r4(tens):
        if check_sub(tens, 0,0) or check_sub(tens, 1,0) or check_sub(tens, 0,1) or check_sub(tens, 1,1):
            if check(tens):
                return (1,1,0)
            return (1,0,0)
        #return (0,0,0)
    if check_r4(tens):
        return (0,0,1)
    return (0,0,0)

te = time.time()
total = 10000
results = Parallel(n_jobs=4)(delayed(proc)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res, total-res[0] - res[2])
print(time.time() - te)

[16, 16, 4956] 5028
34.05723166465759


In [43]:
def proc(i):
    tens = tl.tensor(np.random.randint(100, 2000000, size=(3,2,2))) * 0.01
    a = check_sub(tens, 0,0) or check_sub(tens, 1,0) or check_sub(tens, 0,1) or check_sub(tens, 1,1)
    b = check(tens)
    c = check_r4(tens)
    ret = [0,0,0,0,0]
    if a:
        ret[0] = 1
    if b:
        ret[1] = 1
    if a and b:
        ret[2] = 1
    if a and c:
        ret[3] = 1
    if c:
        ret[4] = 1
        
    return tuple(ret)

te = time.time()
total = 10000
results = Parallel(n_jobs=4)(delayed(proc)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[2666, 3012, 1329, 0, 5051]
52.13955307006836


In [12]:
tens = rank_tree()
x,y,z1,z2,z3 = symbols("x,y,z1,z2,z3")
#tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
a= 1
M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
       [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
      [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
M1 = Matrix([[M,R]])
#display(Latex(latex(M1)))
l = solve(M1.det()/b)[0]
M_1 = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],l,0,0],
              [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,l,0],
              [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,l]])
M_2 = (M_1,R)
print(l)
N = np.array(M_1, dtype= 'float')
sol = la.solve(N.T @ N, N.T @ np.array(R, dtype='float'))
M_2

0.412953729430721


(Matrix([
 [12788.0,       0,                 1,                 0,                 0],
 [      0,  8036.0, 0.412953729430721,                 0,                 0],
 [20166.0,       0,                 0,                 1,                 0],
 [      0, 12446.0,                 0, 0.412953729430721,                 0],
 [13436.0,       0,                 0,                 0,                 1],
 [      0,  9324.0,                 0,                 0, 0.412953729430721]]),
 Matrix([
 [14774.0],
 [ 9142.0],
 [29499.0],
 [16667.0],
 [16766.0],
 [11326.0]]))

In [13]:
sol

array([[ 1.50995742e+00],
       [ 1.37069235e+00],
       [-4.53533546e+03],
       [-9.50801282e+02],
       [-3.52178786e+03]])

In [7]:
def proc_new(i):
    tens = rank_tree()
    tens = tl.tensor(np.random.randint(1, 100000, size=(3,2,2)))*1.0
    a= 1
    M = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],b,0,0],
                [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,b,0],
                [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,b]])
    R = Matrix([Matrix(tens[0,1]),Matrix(tens[1,1]),Matrix(tens[2,1])])
    M1 = Matrix([[M,R]])
    #display(Latex(latex(M1)))
    sol = solve(M1.det()/b)
    if len(sol) == 0:
        return (0,1)
    l = sol[0]
    M_1 = Matrix([[tens[0][0][0],0,a,0,0],[0,tens[0][0][1],l,0,0],
                  [tens[1][0][0],0,0,a,0],[0,tens[1][0][1],0,l,0],
                  [tens[2][0][0],0,0,0,a],[0,tens[2][0][1],0,0,l]])
    N = np.array(M_1, dtype= 'float')
    sol = la.solve(N.T @ N, N.T @ np.array(R, dtype='float'))
    cond = np.all(sol >= 0) and l >= 0
    cond1 = check_r4(tens)
    ret = [0,0,0]
    if cond and cond1:
        ret[0] = 1
    if cond:
        ret[1] = 1
    if cond1:
        ret[2] = 1
    return tuple(ret)

In [9]:
te = time.time()
total = 10000
results = Parallel(n_jobs=4)(delayed(proc_new)(i) for i in range(total))
res = [sum(x) for x in zip(*results)]
print(res)
print(time.time() - te)

[0, 500, 5071]
262.84266114234924
